# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
In my opinion this is a classification learning problem. Based on given data of the student (labeled data like information about his person, information about course participatio and failures/passes of tests but also studytime and traveltime and so on) we try to classify and "predict the success and failure of students early enough to stage effective interventions" (source: taken out of project description). This means we try to predict classification labels based on the given data. In comparison to classification we try to predcit in regression analysis continous outcomes. We have given a number of predictor variables and continous response outcome (variable) and try to find relationships between those variables. In my opion this does not fit the need to classify the students to clear class labels explaining if he needs intervention or not.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [73]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

print student_data.shape
print student_data.head()
print student_data.shape[0]
print student_data.shape[1]

Student data read successfully!
(395, 31)
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        yes       no       5         3      3    1    1      3        4   
2  ...        yes       no       4         3      2    2    3      3       10   
3  ...        yes      yes       3         2      2    1    1      5        2   
4  ...         no       no       4         3      2    1    2      5        4   

  passed  
0  

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [101]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 # as label is not a feature and so the column count of label will be subtracted

# TODO: Calculate passing students
n_passed = student_data['passed'].str.contains(r'yes').sum()

# TODO: Calculate failing students
n_failed = student_data['passed'].str.contains(r'no').sum()

# TODO: Calculate graduation rate
grad_rate = float ( n_passed )/ n_students * 100


# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [102]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [103]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [104]:
# TODO: Import any additional functionality you may need here
from sklearn.utils import shuffle

random_state=0
X_shuffled, y_shuffled = shuffle(X_all, y_all, random_state=random_state)

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = X_shuffled.iloc[:num_train, :]
X_test = X_shuffled.iloc[num_train:, :]
y_train = y_shuffled.iloc[:num_train]
y_test = y_shuffled.iloc[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

First i wouldlike to state out want we know about the given data. These statements are applied to the following answers in section model evaluation:
* We have not that many training and testing points, in total about 400 data points.
* Related to the given count of datapoints we have quite a high dimensionality of features, about 30 in total (I am not yet that experienced in big data sets and machine learning at all. I can imagine that we have datasets with thousands and thousands of labels available but i feel that compared to 395 datapoints 30 lables seems quite many to me).
Regarding the amount and ratio of the given data points and available features we need to consider the Bias–variance tradeoff, meaning that we need to choose models that are robust to a high bias and reduce the risk of high variance.
* We can't assume that the whole data is linearly seperable
* We don't know if features are dependent from each other
* We have a mixed set of features space meaning that we have binary fetures, categoriccal features and continous features
* We don't know if we have outliers
* So we have no deep knowledge about the domain

Perhaps we could answer above statements differently if we would take a deep look into the data and use graphs, scatterplots and other visualization techniues to show the structure of the data. But i would like to use the models to get some feeling if we can conclude on the data structure.


Model Evaluation
* Gaussian Naive Bayes
    * <b>Real-world Application:</b><br />
Naive Bayes can be used for biomedical classification tasks. On the 2009 IEEE International Conference on Bioinformatics and Bioengineering it was shown that Naive Bayes Classifier can be used to predict a target class for unseen biomedical examples
      (https://bi.snu.ac.kr/Publications/Conferences/International/BIBE09_AESNB_MSLee.pdf)
      
    * <b>What are the strengths of the model; when does it perform well:</b>
      * It is very easy to implement and always a good start. that is why i have choosen Navive Bayes as onde model. 
      * According to what we have learned so far naive Bayes works well on big feature spaces
      * It is very efficient and fast
      * It can (not necessarily) work very well on small data sets
      * It performs very well when the features are independent from each other
      * When we don't need to interprete te model
      * Hyperparameter tuning as not complex
      
    * <b>What are the weaknesses of the model; when does it perform poorly:</b>
        * Naive Bayes has the assumption that features are strongly independent from each other. This could lead to a poor performance (not necessarily) of the model. As we don't know the dependence/independece of the dataset features we have to evaluate the model how well it performs.
        * Naive Bayes can't handle continous featues quite good as due to simplification we have the risk to throw important information away
   
    * <b>What makes this model a good candidate for the problem, given what you know about the data</b><br />
    As Naive Bayes is quite easy to implement it looks like a good start for me to evaluate the dataset and feature space. As mentioned above due to the pros and and especially the cons using Naive Bayes i can get an impression how the the model behaves and how the data might be stuctured.
   
   
* Ensemble Methods (AdaBoost)
    * <b>Real-world Application:</b><br />
    AdaBoost is proposed to be used for Face Recognition by the Microsoft Research Center China (http://www.face-rec.org/algorithms/Boosting-Ensemble/RATFG-RTS01Guo.pdf)
    
    * <b>What are the strengths of the model; when does it perform well:</b>
        * There is no need to have a deep domain knowledge for Adaboost (but not that bad to choose starting weights)
        * It has a good generalization performance as we can combine different classifiers
        * Also by combining the weak classifiers we can handle the Bias–variance tradeoff better than one single classifier alone
        * When do not expect many outliers (refer to disadvantages)
        * When we have enough data points (refer to disadvantages)
        * Hyperparameter tuning as not complex
    
    * <b>What are the weaknesses of the model; when does it perform poorly:</b>
        * The model is weak to noise and outliers
        * The model tends to overfit if we have too complex or too weak learners. The performance of the weak leaner should be better than random guessing and < 0.5 generalization error, but not too high performance -> weak learning requirement)
        * We need enough data points so that the weak learning requirement can be satifsfied
    
    * <b>What makes this model a good candidate for the problem, given what you know about the data</b><br />
        * As I do not have deep domain knowledge and the risk of having high variance or high bias is quite high the AdaBoost model can help us to improve the generalization of new data points by combining weak learners. I want to see how it performs compared to Naive Bayes and if my assumption, that it should perform better than Naive Bayes, is correct.
        
        
* KNeighbors
    * <b>Real-world Application:</b><br />
    KNN can be used to classify flowers (http://www.ijcaonline.org/rtippr/number1/SPE95T.pdf)
    It also could be used for predicting economic events (http://www.ijera.com/papers/Vol3_issue5/DI35605610.pdf)
    
    * <b>What are the strengths of the model; when does it perform well:</b>
        * As there is no traiing needed but computational time is used on prediction KNN can be used on smaller datasets
        * KNN can handle diiferent types of data very well
        * KNN is robust to noisy data
        * KNN expects that the data has locality and smoothness. Data points that are close to one another in distance are expected to have similar value
        * It is good if we don't need to interpret a model
    
    * <b>What are the weaknesses of the model; when does it perform poorly:</b>
        * KNN suffers from the Curse of Dimensinality in a special way. As more features need exponential more data we would also need very high computation effort for prediction on big datasets. So that is why for KNN a redcution of dimensionality is important
        * Hyperparameter tuning is complex as this has a great impact on performance 
    * <b>What makes this model a good candidate for the problem, given what you know about the data</b><br />
        * As KNN does not consider the importance of the features. It also does not consider dependency of the features. As it can handle different types of data and also does not "care" about lienar separability i assume that KNN would be a good test to look how performance is on the given dataset. AS KNN as a complete different approach (lazy learner) as the other classiefiers i want to see performance compared to the other candidates. As it is computatiola expensive on bigger datasets it looks as a good decision for me to try out KNN.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [105]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [111]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
# from sklearn import model_B
from sklearn.ensemble import AdaBoostClassifier
# from sklearn import model_C
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = AdaBoostClassifier(random_state=3)
clf_C = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train.iloc[:100, :]
y_train_100 = y_train.iloc[:100]

X_train_200 = X_train.iloc[:200, :]
y_train_200 = y_train.iloc[:200]

X_train_300 = X_train.iloc[:300, :]
y_train_300 = y_train.iloc[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8143.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7681.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0008 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.7969.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7385.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0008 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8108.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7121.

AdaBoostClassifier: 

Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.0913 seconds
Made predictions in 0.0039 seconds.
F1 score for training set: 0.9449.
Made predictions in 0.0038 seconds.
F1 score for test set: 0.7660.
Training a AdaBoostClassifier usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0013           |          0.0003        |      0.8143      |     0.7681      |
| 200               |        0.0007           |          0.0003        |      0.7969      |     0.7385      |
| 300               |        0.0009           |          0.0002        |      0.8108      |     0.7121      |

** Classifer 2 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0889           |          0.0038        |      0.9449      |     0.7660      |
| 200               |        0.0898           |          0.0039        |      0.8594      |     0.7826      |
| 300               |        0.0989           |          0.0038        |      0.8454      |     0.7778      |

** Classifer 3 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0004           |         0.0008         |       0.8209     |     0.7808      |
| 200               |        0.0005           |         0.0011         |       0.8090     |     0.7972      |
| 300               |        0.0006           |         0.0015         |       0.8599     |     0.7832      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Performance of all models does not vary too much. In my opiniopn the KNeighborsClassifier perfoms best in this comparison. It is true that KNN has a higher test prediction time compared to the other models as KNN has its computation efforst on testing and not on training. But regarding the fact that we only have 95 and not thousands of thousands test data points to be comuted i would prefer the KNN at this point. It has also the best available test scores and the lowest variance between training an test performance as well.

I would also suggest that KNN is also in general the model regarding available data, limited resources, cost, and performance.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
The KNearestNeighbor model is called a lazy learner model. This is due to the fact that the model is not trained at the beginning and new data is categorized. Instead it is learning "on-the-fly" meaning, that it compares the new data point with the given data at the moment it comes up. This means that a data point is classified by a majority vote of its neighbors, with that data point being assigned to the class mossly occuring on it's nearest neighbors. The count of nearest neighbors to be compared with can defined.

Regarding our given data we could say, that the model will classify students according to their neighborhood for regarded features. E.g. taking only two features into account, freetime and travel time, we would classify a new student according to other students that has similar freetime and travel time, so  these students would be the nearest neighbor related to those two features. 

This has an effect on memory usage as all data needs to be stored in the memory. It also has effects on the computing time as the classification is done when new data comes up. The advantage on the other side is that additional training data is easy to fit in. So the model is very flexible to extension and not exhausting new training needs to be done if we extend the available data points.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [112]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = [
    {'n_neighbors': [1, 5, 10, 20], 'algorithm': ['ball_tree']},
    {'n_neighbors': [1, 5, 10, 20], 'algorithm': ['brute']},
 ]

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf,
                       param_grid=parameters,
                       scoring=f1_scorer,
                       cv=10)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0047 seconds.
Tuned model has a training F1 score of 0.8204.
Made predictions in 0.0019 seconds.
Tuned model has a testing F1 score of 0.8205.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
Compared to the untuned model it has a slighly smaller training score but a slightly better testing score. The differences using the parameters i have choosen are marginal so more parameter tuning would be needed to get better results.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.